In [1]:
!python -m spacy download ru_core_news_sm

     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.3 MB 1.4 MB/s eta 0:00:12
     --------------------------------------- 0.1/15.3 MB 656.4 kB/s eta 0:00:24
     --------------------------------------- 0.1/15.3 MB 751.6 kB/s eta 0:00:21
     --------------------------------------- 0.2/15.3 MB 766.6 kB/s eta 0:00:20
     --------------------------------------- 0.2/15.3 MB 700.2 kB/s eta 0:00:22
      -------------------------------------- 0.2/15.3 MB 811.5 kB/s eta 0:00:19
      -------------------------------------- 0.2/15.3 MB 722.1 kB/s eta 0:00:21
      -------------------------------------- 0.3/15.3 MB 842.9 kB/s eta 0:00:18
      -------------------------------------- 0.3/15.3 MB 787.7 kB/s eta 0:00:19
      -------------------------------------- 0.4/15.3 MB 791.2 kB/s eta 0:00:19
     - ------------------------------------- 0.4/15.3 MB 846.3 kB/s eta 0:00:18
     - ------------------------------------- 0.4/

In [2]:
import spacy
import pandas as pd

from tqdm import tqdm
from spacy.tokens import DocBin

nlp = spacy.load("ru_core_news_sm")

In [3]:
df = pd.read_json('C:\\Users\\slava\\PycharmProjects\\NLP\\kinopoisk.jsonl', encoding="utf8", lines=True)

In [4]:
df = df.sample(frac=0.5)

In [5]:
df['grade3'].value_counts()

grade3
Good       13621
Bad         2377
Neutral     2298
Name: count, dtype: int64

In [6]:
df = df[df['grade3'] != 'Neutral']

In [7]:
new_df = df[['grade3', 'content']]

In [8]:
data = [tuple(new_df.iloc[i].values) for i in range(new_df.shape[0])]

In [9]:
df['grade3'].value_counts()

grade3
Good    13621
Bad      2377
Name: count, dtype: int64

In [10]:
train_data = data[:12000]
valid_data = data[4049:]
print(data)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [11]:
def make_docs(data):
    """
    this will take a list of texts and labels
    and transform them in spacy documents
    data: list(tuple(text, label))
    returns: List(spacy.Doc.doc)
    """
    docs = []
    # nlp.pipe([texts]) is way faster than running
    # nlp(text) for each text
    # as_tuples allows us to pass in a tuple,
    # the first one is treated as text
    # the second one will get returned as it is.
    # a = tqdm(nlp.pipe(data, as_tuples=True), total = len(data))
    for doc, label in tqdm(nlp.pipe(data, as_tuples=True), total = len(data)):
        if label == 'Bad':
            doc.cats["Good"] = 0
            doc.cats["Bad"] = 1
        else:
            doc.cats["Good"] = 1
            doc.cats["Bad"] = 0
        # we need to set the (text)cat(egory) for each document
        #doc.cats["positive"] = label
        # put them into a nice list
        docs.append(doc)
    return docs

In [12]:
# we are so far only interested in the first 5000 reviews
# this will keep the training time short.
# In practice take as much data as you can get.
# you can always reduce it to make the script even faster.
num_texts = 5000
# first we need to transform all the training data
train_docs = make_docs(data[:num_texts])

100%|██████████| 5000/5000 [00:01<00:00, 3252.08it/s]


In [13]:

# then we save it in a binary file to disc
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("train.spacy")
# repeat for validation data
valid_docs = make_docs(valid_data[:num_texts])
doc_bin = DocBin(docs=valid_docs)
doc_bin.to_disk("valid.spacy")

100%|██████████| 5000/5000 [00:01<00:00, 3108.07it/s]


на этом месте мы идем в https://spacy.io/usage/training#quickstart, там настраиваем под себя конфиг (textcat), копируем его руками(!) в base_config.cfg, указываем правильные пути до трейн и вэлид


In [14]:
!python -m spacy train config.cfg --output ./output

[i] Saving to output directory: output
[i] Using CPU

=========================== Initializing pipeline ===========================


Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\slava\PycharmProjects\NLP\venv\Lib\site-packages\spacy\__main__.py", line 4, in <module>
    setup_cli()
  File "C:\Users\slava\PycharmProjects\NLP\venv\Lib\site-packages\spacy\cli\_util.py", line 92, in setup_cli
    command(prog_name=COMMAND)
  File "C:\Users\slava\PycharmProjects\NLP\venv\Lib\site-packages\click\core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\slava\PycharmProjects\NLP\venv\Lib\site-packages\typer\core.py", line 778, in main
    return _main(
           ^^^^^^
  File "C:\Users\slava\PycharmProjects\NLP\venv\Lib\site-packages\typer\core.py", line 216, in _main
    rv = self.invoke(ctx)
         ^^^^^^^^^^^^^^^^
  File "C:\Users\slava\PycharmProjects\NLP\venv\Lib\site-packages\click\core.py", line 1688, in invoke
    return _process_resul

In [15]:
import spacy
# load thebest model from training
nlp = spacy.load("output/model-best")
text = ""
print("type : ‘quit’ to exit")
# predict the sentiment until someone writes quit
while text != "quit":
    text = input("Please enter example input: ")
    doc = nlp(text)
    print(doc.cats)
    if doc.cats['Good'] >.5:
        print(f"the sentiment is positive")
    else:
        print(f"the sentiment is negative")

type : ‘quit’ to exit
{'Good': 0.8297627568244934, 'Bad': 0.17023727297782898}
the sentiment is positive
{'Good': 0.0, 'Bad': 0.0}
the sentiment is negative
{'Good': 0.8297627568244934, 'Bad': 0.17023727297782898}
the sentiment is positive
{'Good': 0.8297627568244934, 'Bad': 0.17023727297782898}
the sentiment is positive
{'Good': 0.8297627568244934, 'Bad': 0.17023727297782898}
the sentiment is positive
{'Good': 0.8297627568244934, 'Bad': 0.17023727297782898}
the sentiment is positive
{'Good': 0.0, 'Bad': 0.0}
the sentiment is negative
{'Good': 0.0, 'Bad': 0.0}
the sentiment is negative
{'Good': 0.8297627568244934, 'Bad': 0.17023727297782898}
the sentiment is positive


In [121]:
!python -m spacy init fill-config config config.cfg

[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [115]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy

[i] Saving to output directory: output
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'textcat', 'trainable_lemmatizer',
'tagger']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS TEXTCAT  LOSS TRAIN...  LOSS TAGGER  CATS_SCORE  LEMMA_ACC  TAG_ACC  SCORE 
---  ------  ------------  ------------  -------------  -----------  ----------  ---------  -------  ------
  0       0          0.00          0.25          50.00        40.50       50.00     100.00   100.00    0.82
  4     200          0.13         26.85          88.86        67.82       50.00     100.00   100.00    0.82
  9     400          0.01          4.72           0.00         1.25       50.00     100.00   100.00    0.82
 16     600          0.01          0.57           0.00         1.39       50.00     100.00   100.00    0.82
[!] Aborting an

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\slava\PycharmProjects\NLP\venv\Lib\site-packages\spacy\__main__.py", line 4, in <module>
    setup_cli()
  File "C:\Users\slava\PycharmProjects\NLP\venv\Lib\site-packages\spacy\cli\_util.py", line 92, in setup_cli
    command(prog_name=COMMAND)
  File "C:\Users\slava\PycharmProjects\NLP\venv\Lib\site-packages\click\core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\slava\PycharmProjects\NLP\venv\Lib\site-packages\typer\core.py", line 778, in main
    return _main(
           ^^^^^^
  File "C:\Users\slava\PycharmProjects\NLP\venv\Lib\site-packages\typer\core.py", line 216, in _main
    rv = self.invoke(ctx)
         ^^^^^^^^^^^^^^^^
  File "C:\Users\slava\PycharmProjects\NLP\venv\Lib\site-packages\click\core.py", line 1688, in invoke
    return _process_resul

^C
